# Imported packages for use in Project

In [6]:
import yfinance as yf
import pandas as pd
import numpy as np

In [28]:
# Funkce pro získání dat
# Aktualizovaný seznam tickerů podle nového seznamu
tickers = [
    'BABA', 'CVS', 'VOW3.DE', '2318.HK', 'BAYN.DE', 'WBD',
    'CEZ.PR', 'EEFT', 'SOFI', 'DIDY', 'BIDU', 'RYAAY',
    'P911.DE', 'HAL', 'PFE', 'EVO.ST', 'TUI1.DE', 'APPL'
]

def calculate_intrinsic_values_with_fcf(tickers):
    data = []
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            stock_info = stock.info

            # Získání FCF a růstu
            fcf = stock_info.get("freeCashflow", None)
            growth_rate = stock_info.get("earningsGrowth", 0.03)  # Použít 3 %, pokud není dostupné

            # Oprava FCF: Ignorovat záporné nebo nulové hodnoty
            if fcf is None or fcf <= 0:
                print(f"{ticker}: FCF není platné (FCF = {fcf}), přeskočeno.")
                continue

            # Omezit FCF na realistickou hodnotu
            if fcf > 10_000_000_000:  # Například 10 miliard
                print(f"{ticker}: FCF příliš vysoké ({fcf}), omezeno na 10 miliard.")
                fcf = 10_000_000_000

            # Oprava růstu: Použít minimální konzervativní růst
            if growth_rate is None or growth_rate < 0:
                print(f"{ticker}: Růst není platný (Growth Rate = {growth_rate}), nastavena hodnota 3 %.")
                growth_rate = 0.03

            # Diskontní míra a terminální růst
            discount_rate = 0.10  # 10 %
            terminal_growth = 0.02  # 2 %

            # Výpočet vnitřní hodnoty
            intrinsic_val = intrinsic_value(fcf, growth_rate, discount_rate, terminal_growth)

            # Aktuální cena akcie
            current_price = stock.history(period="1d")['Close'].iloc[-1]

            # Uložit výstup
            data.append({
                "Ticker": ticker,
                "Current Price": current_price,
                "Intrinsic Value": intrinsic_val,
                "Undervalued": current_price < intrinsic_val
            })
        except Exception as e:
            print(f"Chyba u {ticker}: {e}")
            continue
    return pd.DataFrame(data)

# Výpočet vnitřních hodnot
intrinsic_data = calculate_intrinsic_values_with_fcf(tickers)

# Zobrazení výsledků
print("Výsledky vnitřních hodnot:")
print(intrinsic_data)

# Uložení výsledků do CSV
intrinsic_data.to_csv("intrinsic_values_with_fcf.csv", index=False)



BABA: FCF příliš vysoké (142874492928), omezeno na 10 miliard.
BABA: Růst není platný (Growth Rate = -0.257), nastavena hodnota 3 %.
CVS: Růst není platný (Growth Rate = -0.961), nastavena hodnota 3 %.
VOW3.DE: FCF příliš vysoké (23552440320), omezeno na 10 miliard.
VOW3.DE: Růst není platný (Growth Rate = -0.688), nastavena hodnota 3 %.
2318.HK: FCF není platné (FCF = -489514500096), přeskočeno.
WBD: FCF příliš vysoké (20279250944), omezeno na 10 miliard.
CEZ.PR: FCF příliš vysoké (93132750848), omezeno na 10 miliard.
CEZ.PR: Růst není platný (Growth Rate = -0.686), nastavena hodnota 3 %.
SOFI: FCF není platné (FCF = None), přeskočeno.


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DIDY?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DIDY&crumb=EO99hzymszV


DIDY: FCF není platné (FCF = None), přeskočeno.
BIDU: FCF příliš vysoké (27584374784), omezeno na 10 miliard.
RYAAY: Růst není platný (Growth Rate = -0.03), nastavena hodnota 3 %.
P911.DE: Růst není platný (Growth Rate = -0.479), nastavena hodnota 3 %.
HAL: Růst není platný (Growth Rate = -0.18), nastavena hodnota 3 %.
PFE: FCF příliš vysoké (11420749824), omezeno na 10 miliard.
APPL: FCF není platné (FCF = None), přeskočeno.
Výsledky vnitřních hodnot:
     Ticker  Current Price  Intrinsic Value  Undervalued
0      BABA      83.129997     1.330036e+11         True
1       CVS      58.009998     2.603711e+10         True
2   VOW3.DE      81.800003     1.330036e+11         True
3   BAYN.DE      19.802000     4.429019e+10         True
4       WBD      10.110000     1.330036e+11         True
5    CEZ.PR     961.500000     1.330036e+11         True
6      EEFT     105.510002     3.670758e+10         True
7      BIDU      80.330002     1.519979e+11         True
8     RYAAY      42.980000    

In [25]:
print("FCF a růst pro každou společnost:")
for ticker in tickers:
    stock = yf.Ticker(ticker)
    stock_info = stock.info
    print(f"{ticker}: FCF = {stock_info.get('freeCashflow')}, Growth Rate = {stock_info.get('earningsGrowth')}")
future_fcf = [fcf * (1 + growth_rate)**t for t in range(1, 6)]
terminal_value = future_fcf[-1] * (1 + terminal_growth) / (discount_rate - terminal_growth)
print(f"Future FCF: {future_fcf}, Terminal Value: {terminal_value}")


FCF a růst pro každou společnost:
BABA: FCF = 142874492928, Growth Rate = -0.257
CVS: FCF = 1957624960, Growth Rate = -0.961
VOW3.DE: FCF = 23552440320, Growth Rate = -0.688
2318.HK: FCF = -489514500096, Growth Rate = 1.506
BAYN.DE: FCF = 3329999872, Growth Rate = None
WBD: FCF = 20279250944, Growth Rate = None
CEZ.PR: FCF = 93132750848, Growth Rate = -0.686
EEFT: FCF = 400375008, Growth Rate = 0.569
SOFI: FCF = None, Growth Rate = None


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DIDY?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DIDY&crumb=EO99hzymszV


DIDY: FCF = None, Growth Rate = None
BIDU: FCF = 27584374784, Growth Rate = 0.062
RYAAY: FCF = 1269650048, Growth Rate = -0.03
P911.DE: FCF = 5996375040, Growth Rate = -0.479
HAL: FCF = 1845374976, Growth Rate = -0.18
PFE: FCF = 11420749824, Growth Rate = None
EVO.ST: FCF = 871645248, Growth Rate = 0.246
TUI1.DE: FCF = 367137504, Growth Rate = 0.479
Future FCF: [1050000.0, 1102500.0, 1157625.0000000002, 1215506.2500000002, 1276281.5625000005], Terminal Value: 18779571.562500007


## Filtering based on Metrics


In [22]:
# Normalizace hodnot
stock_data['PEG_Score'] = 1 / (stock_data['PEG'] + 1).replace([np.inf, -np.inf], np.nan).fillna(0)  # Nižší PEG = vyšší skóre
stock_data['ROE_Score'] = stock_data['ROE'].fillna(0)  # Vyšší ROE = lepší
stock_data['Volatility_Score'] = stock_data['Volatility'].fillna(0)  # Vyšší volatilita = lepší pro swing
stock_data['Debt_Score'] = 1 / (stock_data['Debt/Equity'] + 1).replace([np.inf, -np.inf], np.nan).fillna(0)  # Nižší zadlužení = lepší

# Celkové skóre
stock_data['Total_Score'] = (
    0.4 * stock_data['PEG_Score'] +  # Váha pro PEG
    0.3 * stock_data['ROE_Score'] +  # Váha pro ROE
    0.2 * stock_data['Volatility_Score'] +  # Váha pro volatilitu
    0.1 * stock_data['Debt_Score']  # Váha pro zadlužení
)

# Výběr nejlepších akcií
top_stocks = stock_data.nlargest(5, 'Total_Score')  # Top 5 akcií
print("\nTop akcie podle skóre:")
print(top_stocks)



Top akcie podle skóre:
     Ticker         P/E       P/B      ROE  EV/EBITDA  Debt/Equity  \
4   BAYN.DE         NaN  0.636270 -0.02727      5.526      134.104   
5       WBD         NaN  0.706252 -0.27295      8.780      111.154   
8      SOFI  130.000000  2.763018  0.03725        NaN       54.394   
13      PFE   34.200000  1.574779  0.04538     11.087       73.408   
15  TUI1.DE    4.242286  8.977025  0.72203      5.950      374.158   

    Dividend Yield  EPS Growth       PEG  Volatility     Liquidity  PEG_Score  \
4           0.0056         NaN       NaN    0.380039  3.372206e+06        inf   
5              NaN         NaN       NaN    0.510061  3.078138e+07        inf   
8              NaN         NaN       NaN    0.490405  4.427764e+07        inf   
13          0.0655         NaN       NaN    0.220339  3.533351e+07        inf   
15             NaN       0.479  8.856546    0.351753  3.669090e+06   0.101455   

    ROE_Score  Volatility_Score  Debt_Score  Total_Score  
4    -0.0

# Disconted Cash Flow

In [23]:
def intrinsic_value(fcf, growth_rate, discount_rate, terminal_growth, years=5):
    # Odhad budoucích peněžních toků
    future_fcf = [fcf * (1 + growth_rate)**t for t in range(1, years + 1)]

    # Současná hodnota těchto toků
    present_value_fcf = [fcf / (1 + discount_rate)**t for t, fcf in enumerate(future_fcf, start=1)]

    # Terminal value
    terminal_value = future_fcf[-1] * (1 + terminal_growth) / (discount_rate - terminal_growth)
    present_terminal_value = terminal_value / (1 + discount_rate)**years

    # Celková vnitřní hodnota
    return sum(present_value_fcf) + present_terminal_value

# Příklad výpočtu
fcf = 1_000_000  # Současný volný peněžní tok (v CZK)
growth_rate = 0.05  # Roční růst FCF
discount_rate = 0.10  # Diskontní míra
terminal_growth = 0.03  # Růst terminální hodnoty

value = intrinsic_value(fcf, growth_rate, discount_rate, terminal_growth)
print(f"Vnitřní hodnota společnosti: {value:.2f} CZK")


Vnitřní hodnota společnosti: 16018757.26 CZK
